In [114]:
import pandas as pd
import numpy as np
import glob
import time
runflag=1

In [115]:
data_dir="workshop_dataset/workshop_dataset1/";
all_files = glob.glob(data_dir+"*.json")
total_files = len(all_files)

In [116]:
df=pd.read_json(all_files[0],orient='index',
                convert_dates=False, #dont convert columns to dates 
                convert_axes=False, #dont convert index to dates
        )
# print(df.info())
string = pd.io.sql.get_schema(df, 'twitter_table')
# print(string)

In [161]:
# Test the data for consistencies. 

# Check Identical columns in all files and
# Note Record Counts to tally with cassandra counts later
if(runflag == 1):
    i=0
    df_shapes=[]
    all_records = pd.DataFrame()
    time_taken = 0
    prev_time = time.time()
    for filename in all_files:
        df=pd.read_json(filename,orient='index',convert_dates=False,convert_axes=False)    
        df_shapes.append(df.shape)
        curr_set = set(df)
        if(i>0):
            all_records = pd.concat([all_records,df])
            if(prev_set != curr_set):
                runflag=1
                print(filename, "New columns found!", curr_set - prev_set)
                break
        else:
            all_records=df
        prev_set = curr_set
        i+=1
        runflag=0
    time_taken= time.time()-prev_time

# Rename columns that are keywords
all_records = all_records.rename(columns={'date':'tweet_date','datetime':'tweet_datetime'})
# convert dates to datetime objects
all_records['tweet_date'] = pd.to_datetime(all_records['tweet_date'])
all_records['tweet_datetime'] = pd.to_datetime(all_records['tweet_datetime'])
# convert lists to str as the python driver doesn't do it before utf-encode 
list_columns=['hashtags', 'keywords_processed_list', 'mentions', 'url_list']
for c in list_columns:
    all_records[c]=all_records[c].astype(str)
# floats to ints coz pandas uses float which supports NoneTypes too, ints can't support this in pandas!
# float_columns=['quoted_source_id', 'replyto_source_id', 'retweet_source_id']
# for c in list_columns:
#     all_records[c]=all_records[c].astype(int)
# Changing Cassandra columns to float as NULL support required.
total_records= sum([ x for x,y in df_shapes])
total_records2= all_records.shape[0]
if(total_records!=total_records2):
    print("Error: Something's wrong with dataset, total counts not matching!",total_records,total_records2)
if(runflag==0):
    print("Done scanning %d files with total of %d records(%.2f records/file), in %d seconds(%.2f files/sec)" %
          (total_files, total_records, float(total_records)/total_files,time_taken, float(total_files)/time_taken))

# Ensure data types are correct in the dataframe-


Error: Something's wrong with dataset, total counts not matching! 111543 5
Done scanning 113 files with total of 111543 records(987.11 records/file), in 35 seconds(3.20 files/sec)


In [ ]:
all_records = all_records.tail(1000)


In [198]:
from cassandra.query import UNSET_VALUE

#debuggin
all_records = all_records.tail(1000)
columns = all_records.columns.values # or list(df)
insertrecords = all_records.as_matrix()
# Handling NULLs -
# https://www.datastax.com/dev/blog/python-driver-2-6-0-rc1-with-cassandra-2-2-features
def handleNullInts(row):
    for i,val in enumerate(row):
        if(type(val)==float):
            if np.isnan(val):
                print("None Set for index",i,val)
                row[i]= UNSET_VALUE 
            else:
                row[i]= int(row[i])
    print('')

for record in insertrecords:
    handleNullInts(record)

#     col=columns[i]
#     print(col,"\t", type(r), "\t", r)
# print(insertrecords[0])


None Set for index 14 nan
None Set for index 16 nan

None Set for index 16 nan
None Set for index 18 nan

None Set for index 14 nan
None Set for index 16 nan

None Set for index 14 nan
None Set for index 16 nan
None Set for index 18 nan

None Set for index 14 nan
None Set for index 16 nan



In [200]:
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent, execute_concurrent_with_args

cluster = Cluster(['127.0.0.1'])

session = cluster.connect('twitterdb') # keyspace name should be lowercase
statement = session.prepare("insert into twitter_table (author, author_id, author_profile_image, author_screen_name, tweet_date, tweet_datetime, hashtags, keywords_processed_list, lang, like_count, location, media_list, mentions, quote_count, quoted_source_id, reply_count, replyto_source_id, retweet_count, retweet_source_id, sentiment, tid, tweet_text, type, url_list, verified) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")
# Note that it will overwrite data without error (by design)
# session.execute(statement,insertrecords[0])
execute_concurrent_with_args(session, statement, insertrecords, concurrency=50)

session.shutdown(); # for use by cqlsh
cluster.shutdown();